## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database_Copy.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,76.73,80,90,5.75,light rain
1,1,Ushuaia,AR,-54.8000,-68.3000,42.46,70,20,5.75,few clouds
2,2,Kavieng,PG,-2.5744,150.7967,80.98,75,12,7.54,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,69.08,63,100,22.84,overcast clouds
4,4,Khatanga,RU,71.9667,102.5000,65.59,63,89,8.50,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,76.73,80,90,5.75,light rain
2,2,Kavieng,PG,-2.5744,150.7967,80.98,75,12,7.54,few clouds
11,11,Albany,US,42.6001,-73.9662,76.89,64,74,1.99,broken clouds
12,12,Villa Rica,US,33.7321,-84.9191,88.12,84,42,9.93,scattered clouds
19,19,Tias,ES,28.9611,-13.6450,76.68,53,20,29.93,few clouds
21,21,Hithadhoo,MV,-0.6000,73.0833,81.00,73,100,4.47,overcast clouds
23,23,Bengkulu,ID,-3.8004,102.2655,75.38,91,90,3.87,overcast clouds
24,24,Meulaboh,ID,4.1363,96.1285,75.31,92,100,4.70,overcast clouds
32,32,Dong Hoi,VN,17.4833,106.6000,83.43,75,100,8.16,light rain
34,34,Hasaki,JP,35.7333,140.8333,77.29,89,1,7.87,clear sky


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,76.73,light rain,19.7297,-155.0900,
2,Kavieng,PG,80.98,few clouds,-2.5744,150.7967,
11,Albany,US,76.89,broken clouds,42.6001,-73.9662,
12,Villa Rica,US,88.12,scattered clouds,33.7321,-84.9191,
19,Tias,ES,76.68,few clouds,28.9611,-13.6450,
21,Hithadhoo,MV,81.00,overcast clouds,-0.6000,73.0833,
23,Bengkulu,ID,75.38,overcast clouds,-3.8004,102.2655,
24,Meulaboh,ID,75.31,overcast clouds,4.1363,96.1285,
32,Dong Hoi,VN,83.43,light rain,17.4833,106.6000,
34,Hasaki,JP,77.29,clear sky,35.7333,140.8333,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, SyntaxError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [29]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,76.73,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Kavieng,PG,80.98,few clouds,-2.5744,150.7967,Nusa Island Retreat
12,Villa Rica,US,88.12,scattered clouds,33.7321,-84.9191,Days Inn by Wyndham Villa Rica
19,Tias,ES,76.68,few clouds,28.9611,-13.6450,Hotel Beatriz Playa & SPA
21,Hithadhoo,MV,81.00,overcast clouds,-0.6000,73.0833,Scoop Guest House


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))